# Deploying a model using FastAPI and Docker

Before we can deploy our model, we need to build it. So let's start with that and build a classification model that classifies email as spam or ham. We will then save it using pickle and using it in a fastapi server.

In [ ]:
import pandas as pd
import numpy as np

import re # regex library

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression

import skops.io as sio

## Building the Model

In [ ]:
# Read the Data
data = pd.read_csv('./data/data.csv')

# Text Preprocessing
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text) # Effectively removes HTML markup tags
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    return text

# Train, Test Split
X = data['Message'].apply(preprocessor)
y = data['Category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Training a Logistic Regression Pipeline
tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, 
                        max_features=700, 
                        ngram_range=(1,1))

log_reg_pipe = Pipeline([('vectorizer', tfidf),
                         ('log_reg', LogisticRegression(solver='lbfgs'))])

log_reg_pipe.fit(X_train, y_train)

y_pred = log_reg_pipe.predict(X_test)

print(classification_report(y_test, y_pred))
print('Accuracy: {} %'.format(100 * accuracy_score(y_test, y_pred)))

# Saving the Pipeline
with open('model.bin', 'wb') as f_out:
    sio.dump(log_reg_pipe, f_out)

Now that we have our model/pipeline saved as ```model.bin```, we can continue with deploying our model. There are several ways to do this. ```Flask``` and ```FastAPI``` are two of the more common ones. We will use the fastapi framework to create a fastapi server.
The created fastapi server can be used to deploy our model, so we can call our model with a link to make a prediction.

You can see that code in the ```main.py``` file. Afterwards you can use the Dockerfile to create a Dockerimage which than can be deployed to a Docker registry or run locally as a docker container.

Why you shouldn't use pickle for saving your model: [Link](https://medium.com/ochrona/python-pickle-is-notoriously-insecure-d6651f1974c9). You can use skops or ONNX instead.